In [1]:
import numpy as np

from gurobipy import Model, GRB

# Refinery Optimisation

Bruno M. Pacheco - 16100865

## Problem Definition

The refinery transforms **crude oil** in petrols and fuels to be sold. Thus, we want to **maximize profit** in the daily operation.

<img src="Refinery.png">

In [2]:
m = Model('Refinery')

Academic license - for non-commercial use only - expires 2021-06-12
Using license file /home/bruno/gurobi.lic


### Supply

Crude oil 1 and crude oil 2 are available for production in a fixed rate of 20 000 and 30 000 barrels a day resp.

### Distillation

Splits the two types of crude oil in different subproducts, with a bit of waste, and at most 45 000 barrels of crude oil a day.

|               | Crude Oil 1 (CRA) | Crude Oil 2 (CRB) |
|:--------------|:-----------:|:-----------:|
| (LN) Light naphta  |     0.1     |     0.15    |
| (MN) Medium naphta |     0.2     |     0.25    |
| (HN) Heavy naphta  |     0.2     |     0.18    |
| (LO) Light oil     |     0.12    |     0.08    |
| (HO) Heavy oil     |     0.2     |     0.19    |
| (R)  Residuum      |     0.13    |     0.12    |

### Reforming

Can be applied to **naphtas**, generating reformed gasoline.

|                   | (LNRG) Light naphta | (MNRG) Medium naphta | (HNRG) Heavy naphta |
|-------------------|:------------:|:-------------:|:------------:|
| (RG) Reformed gasoline |      0.6     |      0.52     |     0.45     |

At most 10 000 barrels of naphta can be reformed per day.

### Cracking

Applies to the **refined oils** and has a limit of 8000 barrels of oil a day.

|                  | (LOCGO) Light oil | (HOCGO) Heavy oil |
|------------------|:---------:|:---------:|
| (CO) Cracked oil      |    0.68   |    0.75   |
| (CG) Cracked gasoline |    0.28   |    0.20   |

Cracking can also be applied to **residuum** to produce lube-oil.

|          | (RLBO) Residuum |
|----------|:--------:|
| (LBO) Lube-oil |    0.5   |

The refinery must produce _at least_ 500 barrels and _at most_ 1000 barrels daily.

### Blending

The materials can be blended into:
- Motor fuel (petrol)
- Jet fuel
- Fuel oil

#### Motor fuel

A mixture of naphtas, reformed gasoline and cracked gasoline can generate premium motor fuel (PMF) or regular motor fuel (RMF) depending on its resulting octane number.

If the octane number of the blend (assumed to be linear) is higher than 84, it is sold as _regular_. If it is higher than 94, it is sold as _premium_. Reformed gasoline and cracked gasoline have an octane number of 115 and 105 resp., while the light, medium and heavy naphtas have 90, 80 and 70 octane number resp.

Premium motor fuel production must be at least 40 % that of regular motor fuel.

#### Jet fuel

A mixture of light, heavy, cracked oils and residuum.

The vapour pressure must not exceed 1 kg.cm². It is assumed to blend linearly by volume, being 1.0, 0.6, 1.5 and 0.05 kg.cm² the vapour pressure of light, heavy, cracked oils and residuum resp.

#### Fuel oil

Similar to jet fuel but with a fixed ratio of 10:4:3:1 (total 18 parts) of light oil, cracked oil, heavy oil and residuum must be respected.

### Goal

The fuels, fuel oil and lube-oil are the final products to be sold.

|                          | Pounds per barrel |
|--------------------------|:----------------:|
| (PMF) Premium motor fuel |        7.0       |
| (RMF) Regular motor fuel |        6.0       |
| (JF) Jet fuel            |        4.0       |
| (FO) Fuel oil            |        3.5       |
| (LO) Lube-oil            |        1.5       |

## Constraints

Five types of constraints:
- Availability
- Production
- Continuity
- Capacity
- Quality

### Availability

The input of the whole process, that is the two crude oils. We'll use CRA and CRB for the amount of each oil used.

$$
    CRA \le 20 000 \\
    CRB \le 30 000
$$

In [3]:
cra = m.addVar(lb=0, ub=20000, name='CRA')
crb = m.addVar(lb=0, ub=30000, name='CRB')

m.update()

### Production

These constraints tie the produced products to the input "ingredients".

#### Distillation

The relation between the crude oils consumed and the resulting products is
$$
\begin{bmatrix}
0.1     &     0.15    \\
0.2     &     0.25    \\
0.2     &     0.18    \\
0.12    &     0.08    \\
0.2     &     0.19    \\
0.13    &     0.12 
\end{bmatrix} \begin{bmatrix} CRA \\ CRB \end{bmatrix} = \begin{bmatrix}
LN \\
MN \\
HN \\
LO \\
HO \\
R
\end{bmatrix}
$$

In [4]:
# outputs
ln = m.addVar(lb=0, name='LN')
mn = m.addVar(lb=0, name='MN')
hn = m.addVar(lb=0, name='HN')
lo = m.addVar(lb=0, name='LO')
ho = m.addVar(lb=0, name='HO')
r = m.addVar(lb=0, name='R')

m.addConstr(0.1*cra + 0.15*crb == ln, 'Prod_LN')
m.addConstr(0.2*cra + 0.25*crb == mn, 'Prod_MN')
m.addConstr(0.2*cra + 0.18*crb == hn, 'Prod_HN')
m.addConstr(0.12*cra + 0.08*crb == lo, 'Prod_LO')
m.addConstr(0.2*cra + 0.19*crb == ho, 'Prod_HO')
m.addConstr(0.13*cra + 0.12*crb == r, 'Prod_R')

m.update()

#### Reforming

We denote the amount of each nafta used for reformed gasoline production through the suffix -RG.

We assure the relation between the naftas used for reforming and the reforming gasoline produced $$
0.6 LNRG + 0.52 MNRG + 0.45 HNRG = RG
$$

In [5]:
# inputs
lnrg = m.addVar(lb=0, name='LNRG')
mnrg = m.addVar(lb=0, name='MNRG')
hnrg = m.addVar(lb=0, name='HNRG')

# outputs
rg = m.addVar(lb=0, name='RG')

m.addConstr(0.6*lnrg + 0.52*mnrg + 0.45*hnrg == rg, 'Prod_RG')

m.update()

#### Cracking

As we saw, cracked oil and gasoline are produced from light and heavy oil.
$$
    0.68 LOCGO + 0.75HOCGO = CO \\
    0.28 LOCGO + 0.20HOCGO = CG
$$

In [6]:
# inputs
locgo = m.addVar(lb=0, name='LOCGO')
hocgo = m.addVar(lb=0, name='HOCGO')

# outputs
co = m.addVar(lb=0, name='CO')
cg = m.addVar(lb=0, name='CG')

m.addConstr(0.68*locgo + 0.75*hocgo == co, 'Prod_CO')
m.addConstr(0.28*locgo + 0.20*hocgo == cg, 'Prod_CG')

m.update()

And lube-oil is produced from residuum.
$$
    0.5 RLBO = LBO
$$

In [7]:
# inputs
rlbo = m.addVar(lb=0, name='RLBO')

# output
lbo = m.addVar(lb=0, name='LBO')

m.addConstr(0.5*rlbo == lbo, 'Prod_LBO')

m.update()

#### Blending

From the naftas and the gasolines, one can produce **premium or regular motor fuel**.
$$
    LNPMF + MNPMF + HNPMF + RGPMF + CGPMF = PMF \\
    LNRMF + MNRMF + HNRMF + RGRMF + CGRMF = RMF
$$

In [8]:
# inputs
lnpmf = m.addVar(lb=0, name='LNPMF')
mnpmf = m.addVar(lb=0, name='MNPMF')
hnpmf = m.addVar(lb=0, name='HNPMF')
rgpmf = m.addVar(lb=0, name='RGPMF')
cgpmf = m.addVar(lb=0, name='CGPMF')

# outputs
lnrmf = m.addVar(lb=0, name='LNRMF')
mnrmf = m.addVar(lb=0, name='MNRMF')
hnrmf = m.addVar(lb=0, name='HNRMF')
rgrmf = m.addVar(lb=0, name='RGRMF')
cgrmf = m.addVar(lb=0, name='CGRMF')

pmf = m.addVar(lb=0, name='PMF')
rmf = m.addVar(lb=0, name='RMF')

m.addConstr(lnpmf + mnpmf + hnpmf + rgpmf + cgpmf == pmf, 'Prod_PMF')
m.addConstr(lnrmf + mnrmf + hnrmf + rgrmf + cgrmf == rmf, 'Prod_RMF')

m.update()

Also, there's a lower limit of premium motor fuel production given the amount of regular motor fuel produced.
$$
    PMF \ge 0.4 RMF
$$

In [9]:
m.addConstr(pmf >= 0.4*rmf, 'Prod_PMF_RMF')

m.update()

**Jet fuel** is a mixture of all the oils and residuum.
$$
    LOJF + HOJF + COJF + RJF = JF
$$

In [10]:
# inputs
lojf = m.addVar(lb=0, name='LOJF')
hojf = m.addVar(lb=0, name='HOJF')
cojf = m.addVar(lb=0, name='COJF')
rjf = m.addVar(lb=0, name='RJF')

# output
jf = m.addVar(lb=0, name='JF')

m.addConstr(lojf + hojf + cojf + rjf == jf, 'Prod_JF')

m.update()

Since **fuel oil** is produced through a fixed ratio of light oil, cracked oil, heavy oil and residuum, we would have
$$
    LOFO = \frac{10}{18} FO \\
    COFO = \frac{4}{18} FO \\
    HOFO = \frac{3}{18} FO \\
    RFO = \frac{1}{18} FO \\
$$
But this is not necessary, as we can add this straight into the continuity constraint of each of these variables later on, reducing the amount of variables and constraints.

In [11]:
fo = m.addVar(lb=0, name='FO')

m.update()

### Continuity

These constraints assure that the destinations sum up to the productions.

#### Naftas

Naftas can be used for producing reformed gasoline, premium and regular motor fuel.
$$
    LN = LNRG + LNPMF + LNRMF \\
    MN = MNRG + MNPMF + MNRMF \\
    HN = HNRG + HNPMF + HNRMF
$$

In [12]:
m.addConstr(ln == lnrg + lnpmf + lnrmf, 'Con_LN')
m.addConstr(mn == mnrg + mnpmf + mnrmf, 'Con_MN')
m.addConstr(hn == hnrg + hnpmf + hnrmf, 'Con_HN')

m.update()

#### Oils and residuum

Light and heavy oils are used for cracked oil and gasoline, jet fuel and fuel oil, the latter through a fixed ratio
$$
    LO = LOCGO + LOJF + \frac{10}{18} FO \\
    HO = HOCGO + HOJF + \frac{3}{18} FO \\
$$

Cracked oil is used only for jet fuel and fuel oil
$$
    CO = COJF + \frac{4}{18} FO \\
$$

And residuum is used for lube-oil, jet fuel and fuel oil
$$
    R = RLBO + RJF + \frac{1}{18} FO \\
$$

In [13]:
m.addConstr(lo == locgo + lojf + 0.55*fo, 'Con_LO')
m.addConstr(ho == hocgo + hojf + 0.17*fo, 'Con_HO')

m.addConstr(co == cojf + 0.22*fo, 'Con_CO')

m.addConstr(r == rlbo + rjf + 0.06*fo, 'Con_R')

m.update()

#### Gasolines

They are only used for the motor fuels
$$
    CG = CGPMF + CGRMF \\
    RG = RGPMF + RGRMF
$$

In [14]:
m.addConstr(cg == cgpmf + cgrmf, 'Con_CG')
m.addConstr(rg == rgpmf + rgrmf, 'Con_RG')

m.update()

### Capacity

These constraints dictate the capacity of each process.

#### Distillation

The distillery has a limit of 45 000 barrels of crude oil a day. $$
    CRA + CRB \le 45 000
$$

In [15]:
m.addConstr(cra + crb <= 45000, 'Cap_Dist')

m.update()

#### Reforming

At most 10 000 barrels of naphta can be reformed per day. $$
    LNRG + MNRG + HNRG \le 10000
$$

In [16]:
m.addConstr(lnrg + mnrg + hnrg <= 10000, 'Cap_Ref')

m.update()

#### Cracking

This facility has a limit of 8000 barrels of oil a day. $$
    LOCGO + HOCGO \le 8000
$$

In [17]:
m.addConstr(locgo + hocgo <= 8000, 'Cap_Crack')

m.update()

Regarding lube-oil, _at least_ 500 barrels and _at most_ 1000 barrels must be produced. $$
    LBO \ge 500 \\
    LBO \le 1000
$$

In [18]:
m.addConstr(lbo >= 500, 'Cap_LBO_min')
m.addConstr(lbo <= 1000, 'Cap_LBO_max')

m.update()

### Quality

These are the constraints that control side-aspects of the processes.

#### Motor fuel

First, we must respect the octane number of each type of motor fuel, remembering that reformed gasoline and cracked gasoline have an octane number of 115 and 105 resp. while the light, medium and heavy naphtas have 90, 80 and 70 octane number resp. and we're aiming for 94 for premium and 84 for regular.

$$
    90 LNPMF + 80 MNPMF + 70 HNPMF + 115 RGPMF + 105 CGPMF \ge 94 PMF \\
    90 LNRMF + 80 MNRMF + 70 HNRMF + 115 RGRMF + 105 CGRMF \ge 84 RMF
$$

In [19]:
m.addConstr(90*lnpmf + 80*mnpmf + 70*hnpmf + 115*rgpmf + 105*cgpmf >= 94*pmf, 'Qual_PMF')
m.addConstr(90*lnrmf + 80*mnrmf + 70*hnrmf + 115*rgrmf + 105*cgrmf >= 84*rmf, 'Qual_RMF')

m.update()

#### Jet fuel

Similar to motor fuel, jet fuel has a restriction on its vapour pressure.

$$
    1.0 LOJF + 0.6 HOJF + 1.5 COJF + 0.05 RJF \le JF
$$

In [20]:
m.addConstr(lojf + 0.6*hojf + 1.5*cojf + 0.05*rjf <= jf, 'Qual_JF')

m.update()

## Objective function

Given the prices of the products we can sell, one can state the objective as

$$
    \max 7.0 PMF + 6.0 RMF + 4.0 JF + 3.5 FO + 1.5 LBO
$$

In [21]:
m.setObjective(7*pmf + 6*rmf + 4*jf + 3.5*fo + 1.5*lbo, GRB.MAXIMIZE)

m.update()

<img src="Refinery.png">

## Optimization

In [22]:
%time m.optimize()

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (linux64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 31 rows, 36 columns and 108 nonzeros
Model fingerprint: 0x8ec7e929
Coefficient statistics:
  Matrix range     [5e-02, 1e+02]
  Objective range  [2e+00, 7e+00]
  Bounds range     [2e+04, 3e+04]
  RHS range        [5e+02, 4e+04]
Presolve removed 15 rows and 14 columns
Presolve time: 0.01s
Presolved: 16 rows, 22 columns, 72 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.3000000e+31   5.000000e+29   1.300000e+01      0s
      14    2.1136513e+05   0.000000e+00   0.000000e+00      0s

Solved in 14 iterations and 0.03 seconds
Optimal objective  2.113651348e+05
CPU times: user 32.7 ms, sys: 11.6 ms, total: 44.2 ms
Wall time: 40.3 ms


In [41]:
print('Obj: £%g\n' % m.objVal)

oils = [cra, crb]
ingredients = [ln, mn, hn, lo, ho, r, rg, cg, co]
products = [pmf, rmf, jf, fo, lbo]

n = max(len(oils), len(ingredients), len(products))

oils.extend([None,]*(n-len(oils)))
ingredients.extend([None,]*(n-len(ingredients)))
products.extend([None,]*(n-len(products)))

for oil, ingredient, product in zip(oils, ingredients, products):
    for v in [oil, ingredient, product]:
        p = '%s %g' % (v.varName, v.x) if v is not None else ''
        print(p, end=' '*(30 - len(p)))
    print('')

CRA 15000                     LN 6000                       PMF 6817.78                   
CRB 30000                     MN 10500                      RMF 17044.4                   
                              HN 8400                       JF 15156                      
                              LO 4200                       FO 0                          
                              HO 8700                       LBO 500                       
                              R 5550                                                      
                              RG 2433.09                                                  
                              CG 1936                                                     
                              CO 5706                                                     


### Sensitivity analysis

In [24]:
print(f"Revenue by increasing supply of")
print(f"Crude oil 1 £{cra.RC:.2f}/Barrel")
print(f"Crude oil 2 £{crb.RC:.2f}/Barrel")

Revenue by increasing supply of
Crude oil 1 £0.00/Barrel
Crude oil 2 £0.26/Barrel


In [25]:
print("Capacities\n")

print(f"Distillery: \t£{m.getConstrByName('Cap_Dist').Pi:.2f}/Barrel")
print(f"Reforming: \t£{m.getConstrByName('Cap_Ref').Pi:.2f}/Barrel")
print(f"Cracking: \t£{m.getConstrByName('Cap_Crack').Pi:.2f}/Barrel")

Capacities

Distillery: 	£4.47/Barrel
Reforming: 	£0.00/Barrel
Cracking: 	£0.68/Barrel


In [26]:
print("Lube-oil restrictions\n")

print(f"Decreasing lower limit: £{-m.getConstrByName('Cap_LBO_min').Pi:.2f}/Barrel")
print(f"Increasing upper limit: £{m.getConstrByName('Cap_LBO_max').Pi:.2f}/Barrel")

Lube-oil restrictions

Decreasing lower limit: £6.50/Barrel
Increasing upper limit: £0.00/Barrel


## Questions?

## Thank you!